In [10]:
import cv2
import os

import matplotlib.pyplot as plt
%matplotlib inline

## Static Saliency Detection

In [9]:
image_name = 'players.jpg'
image_path = os.path.join('images/', image_name)
                          
image = cv2.imread(image_path)

In [10]:
saliency = cv2.saliency.StaticSaliencySpectralResidual_create()
(sucess, saliencyMap) = saliency.computeSaliency(image)
saliencyMap = (saliencyMap * 255).astype("uint8")

In [16]:
cv2.imshow("Image", image)
cv2.imshow("Output", saliencyMap)
cv2.waitKey(0)

113

In [23]:
saliency = cv2.saliency.StaticSaliencyFineGrained_create()
(_, saliencyMap) = saliency.computeSaliency(image)

threshMap = cv2.threshold(saliencyMap.astype("uint8"), 0, 255,
    cv2.THRESH_BINARY | cv2.THRESH_OTSU)[1]

cv2.imshow("Image", image)
cv2.imshow("Output", saliencyMap)
cv2.imshow("Thresh", threshMap)
cv2.waitKey(0)

116

## Objectness Saliency Detection

In [11]:
import numpy as np
import random

In [12]:
model_path = 'objectness_trained_model' # Path to the bing model

image_name = 'lawn_mower.jpg'
image_path = os.path.join('images/', image_name)

max_detections = 10

In [13]:
saliency = cv2.saliency.ObjectnessBING_create()
saliency.setTrainingPath(model_path)

image = cv2.imread(image_path)
(_, saliencyMap) = saliency.computeSaliency(image)
numDetections = saliencyMap.shape[0]

In [14]:
rois = []

if not os.path.isdir('output-objectness'):
    os.mkdir('output-objectness')

for i in range(0, min(numDetections, max_detections)):
    
    (startX, startY, endX, endY) = saliencyMap[i].flatten()
    
    rois.append(
        cv2.ereiimage[startY:endY, startX: endX].copy()
    )
    
    output = image.copy()
    color = [random.randint(0, 255) for _ in range(3)]
    cv2.rectangle(output, (startX, startY), (endX, endY), color, 2)
    cv2.imwrite('output-objectness/output-{}-{}.jpg'.format(image_name, i), output)

## Classification using Keras` ResNet-50 pretrained on ImageNet dataset

In [2]:
from tensorflow.keras.applications import ResNet50

from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.applications.resnet import preprocess_input, decode_predictions
from tensorflow.keras.applications import imagenet_utils

In [5]:
model = ResNet50(include_top=True, weights='imagenet')

In [16]:
model.input

<tf.Tensor 'input_3:0' shape=(None, 224, 224, 3) dtype=float32>

In [21]:
rois_np = np.array(rois[0], dtype=np.float32)

In [22]:
model.predict(rois_np)

ValueError: Error when checking input: expected input_3 to have 4 dimensions, but got array with shape (255, 511, 3)